# Modeling Fox's Wake-Up Behavior
## Table of Contents

<a id = 'imports'></a>
## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
%matplotlib inline

<a id = 'read'></a>
## Reading in Data

In [2]:
df_frames = pd.read_csv('../data/fox-falco-fd-frames.csv')
df_frames.head()

,Unnamed: 0,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_trigger_analog,fox_Start,fox_Y,...,nfox_combo_count,nfox_dmg,nfox_direction,nfox_last_hit_by,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
1,1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
2,2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
3,3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
4,4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4


In [3]:
df_frames.drop(columns = ['Unnamed: 0', 'fox_last_hit_by', 'nfox_last_hit_by'], inplace = True)
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_trigger_analog,fox_Start,fox_Y,fox_X,...,nfox_Dpad_Left,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4


## Reducing Noise

Action States 322, 323, and 324 are useless because they are the states of the game starting, so we will drop them.
Planned on removing all frames in which both characters are on the ground because. If `game` is a Game object, then I can access this information with `game.frames[frame_index].ports[active_port_index].leader.post.ground`. The problem is that this information is not tracked for the games recorded at Fight Pitt 9. I believe this information is recorded for more recent versions of Slippi.

In [4]:
# Establish which action state values explain nothing
useless_states = [322, 323, 324]

# Return the rows that are not (~) in the useless-states list for Fox and Falco
df_frames = df_frames.loc[(~df_frames['fox_state'].isin(useless_states)) | (~df_frames['nfox_state'].isin(useless_states))]
df_frames.reset_index(inplace = True, drop = True)
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_trigger_analog,fox_Start,fox_Y,fox_X,...,nfox_Dpad_Left,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-49,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.872425,60.0,29,0.0,4
1,20190406T144505,-48,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.532425,60.0,29,1.0,4
2,20190406T144505,-47,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.022425,60.0,29,2.0,4
3,20190406T144505,-46,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,8.342424,60.0,29,3.0,4
4,20190406T144505,-45,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,7.492424,60.0,29,4.0,4


Each missed tech animation has different values, but can be interpreted the same way. Thanks to these, we can colapse all of these values to the same value.

In [5]:
# Map all missed tech values as the same value
missed_tech = [183, 188, 189, 191, 196, 197]
df_frames['fox_state'] = df_frames['fox_state'].apply(lambda val: val if val not in missed_tech else 999)

In [6]:
# Map [missed tech, tech in place, tech-roll forward, tech-roll backward] to their respective values in the target column
missed_and_techs = [999, 199, 200, 201]
df_frames['target'] = df_frames['fox_state'].apply(lambda val: 0 if val not in missed_and_techs else (1 if val == 999 else (2 if val == 199 else (3 if val == 200 else 4))))

| Target Value |         Meaning         |
|:------------:|:-----------------------:|
|       0      | Not a wake-up situation |
|       1      |       Missed tech       |
|       2      |      Tech in-place      |
|       3      |    Tech-roll forward    |
|       4      |    Tech-roll backward   |

In [7]:
# class imbalance
df_frames['target'].value_counts(normalize = True)

0    0.966377
1    0.020880
2    0.005488
4    0.004415
3    0.002839
Name: target, dtype: float64

In [8]:
df_frames['target'].value_counts()

0    101775
1      2199
2       578
4       465
3       299
Name: target, dtype: int64

The baseline accuracy for our multi-classification RNN is 96.63%.

In [9]:
df_frames.columns

Index(['game_id', 'frame_index', 'fox_cstick_x', 'fox_cstick_y',
       'fox_joystick_x', 'fox_joystick_y', 'fox_trigger_analog', 'fox_Start',
       'fox_Y', 'fox_X', 'fox_B', 'fox_A', 'fox_L', 'fox_R', 'fox_Z',
       'fox_Dpad_Up', 'fox_Dpad_Down', 'fox_Dpad_Right', 'fox_Dpad_Left',
       'fox_combo_count', 'fox_dmg', 'fox_direction', 'fox_position_x',
       'fox_position_y', 'fox_shield', 'fox_state', 'fox_state_age',
       'fox_stocks', 'nfox_cstick_x', 'nfox_cstick_y', 'nfox_joystick_x',
       'nfox_joystick_y', 'nfox_trigger_analog', 'nfox_Start', 'nfox_Y',
       'nfox_X', 'nfox_B', 'nfox_A', 'nfox_L', 'nfox_R', 'nfox_Z',
       'nfox_Dpad_Up', 'nfox_Dpad_Down', 'nfox_Dpad_Right', 'nfox_Dpad_Left',
       'nfox_combo_count', 'nfox_dmg', 'nfox_direction', 'nfox_position_x',
       'nfox_position_y', 'nfox_shield', 'nfox_state', 'nfox_state_age',
       'nfox_stocks', 'target'],
      dtype='object')

In [10]:
# Now we need to categorize these values
df_frames = pd.get_dummies(df_frames, columns = ['fox_state', 'nfox_state', 'fox_direction', 'nfox_direction'])

## Modeling

In [11]:
# Establishing X features and y target column
X = df_frames.drop(columns = ['game_id', 'frame_index', 'target'])
y = to_categorical(df_frames['target'])

In [12]:
X.shape

(105316, 345)

In [13]:
y.shape

(105316, 5)

In [14]:
# Since I cannot set random_state to 20XX, I will set it to 2099
# 20XX is a joke among the among the competitive Melee community
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, random_state = 2099)

In [15]:
# Scaling our data
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [16]:
train_sequences = TimeseriesGenerator(Z_train, y_train,
                                     length = 120, batch_size = 512)

In [17]:
train_sequences[0][0].shape

(512, 120, 345)

In [18]:
test_sequences = TimeseriesGenerator(Z_test, y_test,
                                    length = 120, batch_size = 512)

In [19]:
Z_train.shape

(78987, 345)

In [ ]:
# Constructing the model topology
model = Sequential()

# The input layer
model.add(GRU(337, input_shape = (120, 337), return_sequences = True))
# Send the output of 120 past frames back to this layer
model.add(GRU(337, return_sequences = False))

# A hidden layer of 128 nodes with ReLu activation function
model.add(Dense(128, activation = 'relu'))

# 
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

In [ ]:
hist = model.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('No Regularization');
plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy');
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue');
plt.legend();

In [ ]:
# EARLY STOPPING
model_es = Sequential()

model_es.add(GRU(337, input_shape = (120, 337), return_sequences = True))
model_es.add(GRU(337, return_sequences = False))

model_es.add(Dense(128, activation = 'relu'))

model_es.add(Dense(5, activation = 'softmax'))

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, verbose = 1, mode = 'auto')
model_es.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'], callbacks = early_stop)

hist_es = model_es.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization and Early Stopping')
plt.title('Not expecting much difference')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'red')
plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

plt.legend();

In [ ]:
# L2
model_l2 = Sequential()

model_l2.add(GRU(337, input_shape = (120, 337), return_sequences = True, kernel_regularizer = regularizers.l2(0.01)))
model_l2.add(GRU(337, return_sequences = False, kernel_regularizer = regularizers.l2(0.01)))

model_l2.add(Dense(128, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))

model_l2.add(Dense(5, activation = 'softmax', kernel_regularizer = regularizers.l2(0.01)))

model_l2.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

hist_l2 = model_l2.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization, Early Stopping, and L2 Regularization')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

# plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'darkred')
# plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

plt.plot(hist_l2.history['loss'], label = 'Train Loss L2', color = 'orange')
plt.plot(hist_l2.history['val_loss'], label = 'Test Loss L2', color = 'yellow')

plt.legend();

In [ ]:
# Dropout
model_do = Sequential()

model_do.add(GRU(337, input_shape = (120, 337), return_sequences = True))
model_do.add(GRU(337, return_sequences = False))

model_do.add(Dense(128, activation = 'relu'))
model_do.add(Dropout(0.5))

model_do.add(Dense(5, activation = 'softmax'))

model_do.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

hist_do = model_do.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization, Early Stopping, L2 Regularization, and Dropout')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

# plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'darkred')
# plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

# plt.plot(hist_l2.history['loss'], label = 'Train Loss L2', color = 'orange')
# plt.plot(hist_l2.history['val_loss'], label = 'Test Loss L2', color = 'yellow')

plt.plot(hist_do.history['loss'], label = 'Train Loss Dropout', color = 'darkgreen')
plt.plot(hist_do.history['val_loss'], label = 'Test Loss Dropout', color = 'lightgreen')

plt.legend();

In [ ]:
plt.plot(hist_do.history['loss'], label = 'Train Loss Dropout', color = 'darkgreen')
plt.plot(hist_do.history['val_loss'], label = 'Test Loss Dropout', color = 'lightgreen')